In [ ]:
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [ ]:
# !cp /content/gdrive/MyDrive/sem7btp/RGB.zip /content

In [ ]:
!unzip  /content/gdrive/MyDrive/sem7btp/RGB.zip -d /content/

Archive:  /content/gdrive/MyDrive/sem7btp/RGB.zip
   creating: /content/RGB/
  inflating: /content/RGB/a10_s1_t1_color.avi  
  inflating: /content/RGB/a10_s1_t2_color.avi  
  inflating: /content/RGB/a10_s1_t3_color.avi  
  inflating: /content/RGB/a10_s1_t4_color.avi  
  inflating: /content/RGB/a10_s2_t1_color.avi  
  inflating: /content/RGB/a10_s2_t2_color.avi  
  inflating: /content/RGB/a10_s2_t3_color.avi  
  inflating: /content/RGB/a10_s2_t4_color.avi  
  inflating: /content/RGB/a10_s3_t1_color.avi  
  inflating: /content/RGB/a10_s3_t2_color.avi  
  inflating: /content/RGB/a10_s3_t3_color.avi  
  inflating: /content/RGB/a10_s3_t4_color.avi  
  inflating: /content/RGB/a10_s4_t1_color.avi  
  inflating: /content/RGB/a10_s4_t2_color.avi  
  inflating: /content/RGB/a10_s4_t3_color.avi  
  inflating: /content/RGB/a10_s4_t4_color.avi  
  inflating: /content/RGB/a10_s5_t1_color.avi  
  inflating: /content/RGB/a10_s5_t2_color.avi  
  inflating: /content/RGB/a10_s5_t3_color.avi  
  inflating

In [ ]:
from glob import glob
all_files = []
for i in glob("/content/RGB/*"):
  all_files.append(i)

In [ ]:
all_files[0].split('_')[0].split('/')[-1]

'a20'

In [ ]:
all_tags = [i.split("_")[0].split("/")[-1] for i in all_files]
action_cat = list(set(all_tags))

In [ ]:
import tensorflow as tf
import numpy as np
import os
import glob
import cv2
from sklearn.model_selection import train_test_split

data_dir = '/content/RGB'

# action_cat = ['Abuse', 'Arrest', 'Arson', 'Assault', 'RoadAccidents', 'Burglary', 'Explosion', 'Fighting', 'Robbery', 'Shooting', 'Stealing', 'Shoplifting', 'Vandalism']


In [ ]:
def format_frames(frame, output_size):
  """
    Pad and resize an image from a video.

    Args:
      frame: Image that needs to resized and padded. 
      output_size: Pixel size of the output frame image.

    Return:
      Formatted frame with padding of specified output size.
  """
  frame = tf.image.convert_image_dtype(frame, tf.float32)
  frame = tf.image.resize_with_pad(frame, *output_size)
  return frame


In [ ]:
import random
def extract_frames(video_path,label):
  print(video_path,label)
  n_frames = 20
  output_size = (224,224)
  frame_step = 2
  result = []
  src = cv2.VideoCapture(str(video_path))  

  video_length = src.get(cv2.CAP_PROP_FRAME_COUNT)

  need_length = 1 + (n_frames - 1) * frame_step

  if need_length > video_length:
    start = 0
  else:
    max_start = video_length - need_length
    start = random.randint(0, max_start + 1)

  src.set(cv2.CAP_PROP_POS_FRAMES, start)
  # ret is a boolean indicating whether read was successful, frame is the image itself
  ret, frame = src.read()
  result.append(format_frames(frame, output_size))

  for _ in range(n_frames - 1):
    for _ in range(frame_step):
      ret, frame = src.read()
    if ret:
      frame = format_frames(frame, output_size)
      result.append(frame)
    else:
      result.append(np.zeros_like(result[0]))
  src.release()
  result = np.array(result)[..., [2, 1, 0]]

  label = tf.cast(label, tf.float32)

  return result,label


In [ ]:
for video_path in all_files:
  print(video_path, end=" ")
  print(extract_frames(video_path,1)[0].shape)
  print("works")

In [ ]:
def load_dataset(data_dir, batch_size, img_height, img_width, validation_split=0.2, shuffle=True, seed=42):
    # Get the list of all video files in the dataset folder
    video_files = glob.glob(os.path.join(data_dir,'*.avi'))
    # print(video_files)

    # Create a dictionary mapping each crime category to a unique integer label
    label_map = {category: label for label, category in enumerate(action_cat)}
    print(label_map)

    # Get the labels for all video files
    labels = [label_map[video_file.split("_")[0].split("/")[-1]] for video_file in video_files]
    print(labels)

    # Split the video files into training and validation sets stratified by labels
    train_files, val_files, train_labels, val_labels = train_test_split(video_files, labels, test_size=validation_split, stratify=labels, random_state=seed)
    print(train_files)
    # Define the function to extract frames from each video
    def parse_video(video_file):
        # print(video_file)
    # Load the video using OpenCV
        try:
          cap = cv2.VideoCapture(str(video_file))

          # Extract frames from the video
          frames = []
          while True:
              ret, frame = cap.read()
              if not ret:
                  break
              frame = cv2.resize(frame, (224, 224))
              frames.append(frame)
          cap.release()
        except:
          print("error in", video_file)
          frames = [100]

        # Convert the frames to a numpy array and normalize pixel values
        frames = np.array(frames, dtype=np.float32)
        frames /= 255.0
        # print(frames)
        return frames

    def parse_video_wrapper(video_file, label):
        frames = tf.py_function(parse_video, [video_file], tf.float32)
        return frames, label

    # Create TensorFlow datasets for training and validation sets
    train_dataset = tf.data.Dataset.from_tensor_slices((train_files, train_labels))
    train_dataset = train_dataset.shuffle(len(train_files))
    # train_dataset = train_dataset.map(lambda x, y: (tf.py_function(parse_video, [x], tf.float32), y))
    train_dataset = train_dataset.map(extract_frames)
    train_dataset = train_dataset.batch(batch_size)
    train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)

    val_dataset = tf.data.Dataset.from_tensor_slices((val_files, val_labels))
    val_dataset = val_dataset.shuffle(len(val_files))
    # val_dataset = val_dataset.map(lambda x, y: (tf.py_function(parse_video, [x], tf.float32), y))
    val_dataset = train_dataset.map(extract_frames)
    val_dataset = val_dataset.batch(batch_size)
    val_dataset = val_dataset.prefetch(tf.data.AUTOTUNE)

    return train_dataset, val_dataset

In [ ]:
def frames_from_video_file(video_path, n_frames, output_size = (224,224), frame_step = 2):
  """
    Creates frames from each video file present for each category.

    Args:
      video_path: File path to the video.
      n_frames: Number of frames to be created per video file.
      output_size: Pixel size of the output frame image.

    Return:
      An NumPy array of frames in the shape of (n_frames, height, width, channels).
  """
  # Read each video frame by frame
  result = []
  src = cv2.VideoCapture(str(video_path))  

  video_length = src.get(cv2.CAP_PROP_FRAME_COUNT)

  need_length = 1 + (n_frames - 1) * frame_step

  if need_length > video_length:
    start = 0
  else:
    max_start = video_length - need_length
    start = random.randint(0, max_start + 1)

  src.set(cv2.CAP_PROP_POS_FRAMES, start)
  # ret is a boolean indicating whether read was successful, frame is the image itself
  ret, frame = src.read()
  result.append(format_frames(frame, output_size))

  for _ in range(n_frames - 1):
    for _ in range(frame_step):
      ret, frame = src.read()
    if ret:
      frame = format_frames(frame, output_size)
      result.append(frame)
    else:
      result.append(np.zeros_like(result[0]))
  src.release()
  result = np.array(result)[..., [2, 1, 0]]

  return result


In [ ]:

class FrameGenerator:
  def __init__(self, path, n_frames, training = False):
    """ Returns a set of frames with their associated label. 

      Args:
        path: Video file paths.
        n_frames: Number of frames. 
        training: Boolean to determine if training dataset is being created.
    """
    self.path = path
    self.n_frames = n_frames
    self.training = training
    self.class_names = sorted(set(action_cat))#p.name for p in self.path.iterdir() if p.is_dir()))
    self.class_ids_for_name = dict((name, idx) for idx, name in enumerate(self.class_names))

  def get_files_and_class_names(self):
    video_paths = glob.glob(os.path.join(data_dir,'*.avi'))#list(self.path.glob('/content/RGB/*.avi'))
    # classes = [p.parent.name for p in video_paths] 
    classes = [i.split("_")[0].split("/")[-1] for i in video_paths]
    return video_paths, classes

  def __call__(self):
    video_paths, classes = self.get_files_and_class_names()

    pairs = list(zip(video_paths, classes))

    if self.training:
      random.shuffle(pairs)

    for path, name in pairs:
      video_frames = frames_from_video_file(path, self.n_frames) 
      label = self.class_ids_for_name[name] # Encode labels
      yield video_frames, label

In [ ]:
output_signature = (tf.TensorSpec(shape = (None, None, None, 3), dtype = tf.float32),
                    tf.TensorSpec(shape = (), dtype = tf.int16))
train_ds = tf.data.Dataset.from_generator(FrameGenerator(data_dir, 30, training=True),
                                          output_signature = output_signature)
val_ds = tf.data.Dataset.from_generator(FrameGenerator(data_dir, 30),
                                        output_signature = output_signature)


In [ ]:
next(iter(train_ds))[0].shape

TensorShape([30, 224, 224, 3])

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size = AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size = AUTOTUNE)
train_ds = train_ds.batch(2)
val_ds = val_ds.batch(2)

In [ ]:
next(iter(train_ds))[0].shape

TensorShape([2, 30, 224, 224, 3])

In [ ]:
batch_size = 32
img_height = 224
img_width = 224
validation_split = 0.2

train_dataset, val_dataset = load_dataset(data_dir, batch_size, img_height, img_width, validation_split)

In [ ]:
next(iter(train_dataset))[0]

<tf.Tensor: shape=(32, 0), dtype=float32, numpy=array([], shape=(32, 0), dtype=float32)>

In [ ]:
BATCH_SIZE = 2
AUTO = tf.data.AUTOTUNE
INPUT_SHAPE = (30, 224, 224, 3)
NUM_CLASSES = len(action_cat)

# OPTIMIZER
LEARNING_RATE = 1e-4
WEIGHT_DECAY = 1e-5

# TRAINING
EPOCHS = 20

# TUBELET EMBEDDING
PATCH_SIZE = (8, 8, 8)
NUM_PATCHES = (INPUT_SHAPE[0] // PATCH_SIZE[0]) ** 2

# ViViT ARCHITECTURE
LAYER_NORM_EPS = 1e-6
PROJECTION_DIM = 128
NUM_HEADS = 8
NUM_LAYERS = 8

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

class PositionalEncoder(layers.Layer):
    def __init__(self, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim

    def build(self, input_shape):
        _, num_tokens, _ = input_shape
        self.position_embedding = layers.Embedding(
            input_dim=num_tokens, output_dim=self.embed_dim
        )
        self.positions = tf.range(start=0, limit=num_tokens, delta=1)

    def call(self, encoded_tokens):
        # Encode the positions and add it to the encoded tokens
        encoded_positions = self.position_embedding(self.positions)
        encoded_tokens = encoded_tokens + encoded_positions
        return encoded_tokens

In [ ]:
class TubeletEmbedding(layers.Layer):
    def __init__(self, embed_dim, patch_size, **kwargs):
        super().__init__(**kwargs)
        self.projection = layers.Conv3D(
            filters=embed_dim,
            kernel_size=patch_size,
            strides=patch_size,
            padding="VALID",
        )
        self.flatten = layers.Reshape(target_shape=(-1, embed_dim))

    def call(self, videos):
        projected_patches = self.projection(videos)
        flattened_patches = self.flatten(projected_patches)
        return flattened_patches

In [ ]:
def create_vivit_classifier(
    tubelet_embedder,
    positional_encoder,
    input_shape=INPUT_SHAPE,
    transformer_layers=NUM_LAYERS,
    num_heads=NUM_HEADS,
    embed_dim=PROJECTION_DIM,
    layer_norm_eps=LAYER_NORM_EPS,
    num_classes=NUM_CLASSES,
):
    # Get the input layer
    inputs = layers.Input(shape=input_shape)
    # Create patches.
    patches = tubelet_embedder(inputs)
    # Encode patches.
    encoded_patches = positional_encoder(patches)

    # Create multiple layers of the Transformer block.
    for _ in range(transformer_layers):
        # Layer normalization and MHSA
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim // num_heads, dropout=0.1
        )(x1, x1)

        # Skip connection
        x2 = layers.Add()([attention_output, encoded_patches])

        # Layer Normalization and MLP
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        x3 = keras.Sequential(
            [
                layers.Dense(units=embed_dim * 4, activation=tf.nn.gelu),
                layers.Dense(units=embed_dim, activation=tf.nn.gelu),
            ]
        )(x3)

        # Skip connection
        encoded_patches = layers.Add()([x3, x2])

    # Layer normalization and Global average pooling.
    representation = layers.LayerNormalization(epsilon=layer_norm_eps)(encoded_patches)
    representation = layers.GlobalAvgPool1D()(representation)

    # Classify outputs.
    outputs = layers.Dense(units=num_classes, activation="softmax")(representation)

    # Create the Keras model.
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
def run_experiment():
    # Initialize model
    model = create_vivit_classifier(
        tubelet_embedder=TubeletEmbedding(
            embed_dim=PROJECTION_DIM, patch_size=PATCH_SIZE
        ),
        positional_encoder=PositionalEncoder(embed_dim=PROJECTION_DIM),
    )

    # Compile the model with the optimizer, loss function
    # and the metrics.
    optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
    model.compile(
        optimizer=optimizer,
        loss="sparse_categorical_crossentropy",
        metrics=[
            keras.metrics.SparseCategoricalAccuracy(name="accuracy"),
            keras.metrics.SparseTopKCategoricalAccuracy(5, name="top-5-accuracy"),
        ],
    )

    # Train the model.
    _ = model.fit(train_ds, epochs=EPOCHS, validation_data=val_ds)

    _, accuracy, top_5_accuracy = model.evaluate(val_dataset)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")
    print(f"Test top 5 accuracy: {round(top_5_accuracy * 100, 2)}%")

    return model


model = run_experiment()

Epoch 1/20
    163/Unknown - 5383s 33s/step - loss: 3.5896 - accuracy: 0.0491 - top-5-accuracy: 0.1871